In [1]:
from __future__ import print_function, division
from builtins import range, input

import pandas as pd

In [37]:
#df = pd.read_csv('C:\\JYO DATA\\SAP\\Innovator Challenge\\App_Usage_count.csv')
df = pd.read_csv('C:\\JYO DATA\\SAP\\Innovator Challenge\\App_Usage_count (2).csv')

In [38]:
# create a mapping for Customer Number
unique_cust_ids = set(df.CUSTNO.values)
cust2idx = {}
count = 0
for cust_id in unique_cust_ids:
  cust2idx[cust_id] = count
  count += 1

print("Unique Cust Ids",count)

Unique Cust Ids 3418


In [39]:
# add them to the data frame
df['cust_idx'] = df.apply(lambda row: cust2idx[row.CUSTNO], axis=1)

In [40]:
# create a mapping for Fiori App ids
unique_fiori_ids = set(df.FIORIID.values)
fiori2idx = {}
count = 0
for fiori_id in unique_fiori_ids:
  fiori2idx[fiori_id] = count
  count += 1

print("Unique App Ids",count)

# add them to the data frame
df['fiori_idx'] = df.apply(lambda row: fiori2idx[row.FIORIID], axis=1)

Unique App Ids 1371


In [41]:
# create a mapping for Role Names
unique_role_names = set(df.ROLE_NAME.values)
role2idx = {}
count = 0
for role_name in unique_role_names:
  role2idx[role_name] = count
  count += 1

print("Unique Role Names",count)

# add them to the data frame
df['role_name'] = df.apply(lambda row: role2idx[row.ROLE_NAME], axis=1)

Unique Role Names 200


In [42]:
# create a mapping for Application Components
unique_app_comp = set(df.APP_COMP.values)
comp2idx = {}
count = 0
for app_comp in unique_app_comp:
  comp2idx[app_comp] = count
  count += 1

print("Unique Application Component",count)

# add them to the data frame
df['app_comp'] = df.apply(lambda row: comp2idx[row.APP_COMP], axis=1)

Unique Application Component 34


In [43]:
df.sample(5)

,Unnamed: 0,CALWEEK,CUSTNO,FIORIID,LOB,ROLE_NAME,APP_COMP,USAGE_COUNT,cust_idx,fiori_idx,role_name,app_comp
112113,112113,202025,399446,F4955,Manufacturing,Production Supervisor - Lean Manufacturing,LE,3,2589,1126,32,27
5434,5434,202024,750727,F0250,Manufacturing,Production Planner,PP,10,2205,686,17,22
114978,114978,202025,876264,F3386,Supply Chain,Order Fulfillment Manager (Retail),LO,3,3293,1124,4,19
98741,98741,202024,758706,F1383A,Human Resources,Employee - HR Info,PA,3,2149,863,114,23
213910,213910,202027,761357,F1354,NaN,System Administrator (Solution Manager),SV,1,3213,154,168,16


In [44]:
#Normalization of Usage Count
df['USAGE_COUNT_NORM'] = df.groupby('cust_idx')['USAGE_COUNT'].transform(lambda x: 2 if (max(x)-min(x)) == 0 else ((x-min(x))/(max(x)-min(x)))*4)

In [45]:
df.sample(5)

,Unnamed: 0,CALWEEK,CUSTNO,FIORIID,LOB,ROLE_NAME,APP_COMP,USAGE_COUNT,cust_idx,fiori_idx,role_name,app_comp,USAGE_COUNT_NORM
136227,136227,202024,1421320,F2837,"Commerce, Manufacturing, Sales, Sourcing and P...",Production Planner,PP,3,1784,1198,17,22,1.333333
137535,137535,202022,1520762,F4146,"Marketing, Sales, Sourcing and Procurement, Su...",Contract Manager - Settlement Management,LO,3,2202,597,86,19,0.500000
108063,108063,202023,1573836,F0707,Finance,General Ledger Accountant,FI,3,383,481,112,9,0.800000
61041,61041,202022,203023,F2099,NaN,System Administrator (Solution Manager),SV,4,2669,650,168,16,1.714286
22004,22004,202022,20947,F3099,Human Resources,Employee - HR Info,PA,6,1984,980,114,23,2.400000


### Missing Data Imputation for LOBs using K-NN Algorithm

In [46]:
from sklearn.neighbors import KNeighborsClassifier

In [47]:
#Split the LOBs into columns and create a LOB_nan column
df['LOB_nan'] = df.apply(lambda row: 1 if pd.isna(row['LOB']) else 0, axis=1)

#Create Columns for LOBs by splitting using commas
df2 = pd.concat([df, df['LOB'].str.get_dummies(sep=', ')], axis = 1)

In [48]:
df2.head()

,Unnamed: 0,CALWEEK,CUSTNO,FIORIID,LOB,ROLE_NAME,APP_COMP,USAGE_COUNT,cust_idx,fiori_idx,...,Finance,Human Resources,Manufacturing,Marketing,R&D/Engineering,Sales,Service,Sourcing and Procurement,Supply Chain,Sustainability
0,0,202024,323561,F0402A,"Finance, Sourcing and Procurement, Supply Chain",Purchasing Manager,MM,50,1773,1248,...,1,0,0,0,0,0,0,1,1,0
1,1,202027,323561,F0402A,"Finance, Sourcing and Procurement, Supply Chain",Purchasing Manager,MM,50,1773,1248,...,1,0,0,0,0,0,0,1,1,0
2,2,202027,323561,F0862,NaN,Manager,CA,50,1773,523,...,0,0,0,0,0,0,0,0,0,0
3,3,202024,323561,F0862,NaN,Manager,CA,50,1773,523,...,0,0,0,0,0,0,0,0,0,0
4,4,202024,323561,F5000,Finance,General Ledger Accountant - Intercompany,CA,50,1773,899,...,1,0,0,0,0,0,0,0,0,0


In [49]:
#KNN Start
data = df2.copy()

In [55]:
df2.columns

Index(['Unnamed: 0', 'CALWEEK', 'CUSTNO', 'FIORIID', 'LOB', 'ROLE_NAME',
       'APP_COMP', 'USAGE_COUNT', 'cust_idx', 'fiori_idx', 'role_name',
       'app_comp', 'USAGE_COUNT_NORM', 'LOB_nan', 'Asset Management',
       'Commerce', 'Finance', 'Human Resources', 'Manufacturing', 'Marketing',
       'R&D/Engineering', 'Sales', 'Service', 'Sourcing and Procurement',
       'Supply Chain', 'Sustainability'],
      dtype='object')

In [50]:
#Split into known and unknown LOB data
known, pred = data.drop(data[data['LOB_nan'] == 1].index), data.drop(data[data['LOB_nan'] == 0].index)

In [60]:
#Train based on only Appliction Component and Role Name with target as the list of LOBs
X_train = known[['role_name', 'app_comp']]

print(known.columns[14:])
Y_train = known[known.columns[14:]]

Index(['Asset Management', 'Commerce', 'Finance', 'Human Resources',
       'Manufacturing', 'Marketing', 'R&D/Engineering', 'Sales', 'Service',
       'Sourcing and Procurement', 'Supply Chain', 'Sustainability'],
      dtype='object')


In [61]:
X_pred = pred[['role_name', 'app_comp']]

In [62]:
neighbours = 15

In [63]:
#Train
knn = KNeighborsClassifier(n_neighbors=neighbours)      
knn.fit(X_train, Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=15, p=2,
                     weights='uniform')

In [64]:
#Predict for unknown LOB data
Y_pred = knn.predict(X_pred)

In [67]:
#Save the result to a csv after reverting to Role Names and Cust No
known, pred = df2.drop(df2[df2['LOB_nan'] == 1].index), df2.drop(df2[df2['LOB_nan'] == 0].index)
pred[pred.columns[14:]] = Y_pred

In [68]:
filled_data = known.append(pred, ignore_index = True)
filled_data.sample(5)

,Unnamed: 0,CALWEEK,CUSTNO,FIORIID,LOB,ROLE_NAME,APP_COMP,USAGE_COUNT,cust_idx,fiori_idx,...,Finance,Human Resources,Manufacturing,Marketing,R&D/Engineering,Sales,Service,Sourcing and Procurement,Supply Chain,Sustainability
214999,194960,202026,1194912,F1243,NaN,Other,PLM,1,2931,529,...,1,0,0,0,0,1,1,1,0,0
149999,191379,202027,181985,F0773,"Finance, Sales, Sourcing and Procurement, Supp...",Accounts Receivable Accountant,FI,1,702,1020,...,1,0,0,0,0,1,0,1,1,0
153128,195179,202022,10104,F3173,"Sourcing and Procurement, Supply Chain",Configuration Expert - Business Process Config...,MM,1,785,71,...,0,0,0,0,0,0,0,1,1,0
225158,213476,202026,1121962,F1354,NaN,System Administrator (Solution Manager),SV,1,3270,154,...,1,0,0,0,0,0,0,0,0,0
143861,184121,202027,222289,F1708,"Marketing, Sales, Service",Returns & Refund Clerk,SD,1,434,675,...,0,0,0,1,0,1,1,0,0,0


In [69]:
filled_data.to_csv("C:\\JYO DATA\\SAP\\Innovator Challenge\\editedLong_App_Usage_count.csv")